# Задание по теме «Коллаборативная фильтрация»

## Пакет SURPRISE:

- используйте данные MovieLens 1M,
- можно использовать любые модели из пакета,
- получите RMSE на тестовом сете 0,87 и ниже.


Комментарий преподавателя:
В домашнем задании на датасет 1М может не хватить RAM. Можно сделать на 100K. Качество RMSE предлагаю считать на основе Cross-validation (5 фолдов), а не на отложенном датасете.

In [5]:
from surprise import Dataset, SVD, KNNBasic
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import accuracy
from surprise.model_selection import train_test_split

### Реализовываем загрузку датасета

In [6]:
# Загружаем датасет MovieLens
data = Dataset.load_builtin('ml-1m')


Dataset ml-1m could not be found. Do you want to download it? [Y/n] 

 Y


Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-1m.zip...
Done! Dataset ml-1m has been saved to /root/.surprise_data/ml-1m


### Подбор гиперпараметров для модели SVD

In [7]:
# Разделяем данные на обучающий и тестовый наборы
trainset, testset = train_test_split(data, test_size=0.2)

# Определяем параметры для поиска
param_grid = {
    'n_epochs': [50, 100, 150],
    'lr_all': [0.002, 0.005, 0.007],
    'reg_all': [0.02, 0.04, 0.06, 0.08]
}

# Настраиваем GridSearchCV
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=5, n_jobs=-1)

# Запускаем поиск параметров
gs.fit(data)

# Лучшая комбинация параметров
print("Лучшие параметры:", gs.best_params['rmse'])

# Получим лучшую модель
best_model = gs.best_estimator['rmse']


Лучшие параметры: {'n_epochs': 150, 'lr_all': 0.002, 'reg_all': 0.06}


### Финальная проверка качества на кросс-валидации

In [8]:
# Проверим RMSE на 5 фолдах с лучшей моделью
results = cross_validate(best_model, data, measures=['RMSE'], cv=5, verbose=True)

# Среднее значение RMSE
mean_rmse = sum(results['test_rmse']) / len(results['test_rmse'])
print(f"\nСредний RMSE по 5 фолдам: {mean_rmse:.4f}")

# Тестируем на отложенном наборе данных
best_model.fit(trainset)
predictions = best_model.test(testset)
test_rmse = accuracy.rmse(predictions)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8492  0.8502  0.8533  0.8495  0.8488  0.8502  0.0016  
Fit time          32.15   32.03   30.23   32.82   30.20   31.49   1.07    
Test time         0.81    0.69    0.68    0.78    0.68    0.73    0.06    

Средний RMSE по 5 фолдам: 0.8502
RMSE: 0.8507
